In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split

# Import Numpy
import numpy as np
# For timer
import time
# Import OS
import os

# ==== LOCAL LIBS ====
import ecg_tools_lite as et
import evaluator as eva
import trainer as tr
import models as mod

# Summary-like from Tensorflow
from torchsummary import summary

# wfdb
import wfdb
from wfdb import processing

In [2]:
# What are you trying to do?
create_new_model = False
create_new_res = False

# If create_new_model is True, epochs must have a value
epochs = 10

# Model used:
model_net = mod.cnn_denoiser

# If create_new_model is False, load_model_name must have a value
load_model_name = 'model_2021-3-15_819.pt'
ecg_saved_res = 'res_pt_full_DAE_all_em.npy' #'res_pt_full_2021-3-14_202.npy'

# Normalization option (either option 0 or 1)
norm_option = 2

In [3]:
# For single SNR runs
ecg_noisy = torch.from_numpy(np.load('all_none_fs1024_en6.npy')).cuda().double()
ecg_clean = torch.from_numpy(np.load('all_none_fs1024_e24.npy')).cuda().double()

# Acquire original shape
x = ecg_noisy.shape[0]
y = ecg_noisy.shape[2]
z = ecg_noisy.shape[1]

# Reshape
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_clean = torch.reshape( ecg_clean, (x, y, z))

print(f'New shape: {ecg_noisy.shape}')

# == Normalization technique (Norm from -1 to 1)
ecg_noisy = et.norm_sig( ecg_noisy )
ecg_clean = et.norm_sig( ecg_clean )

New shape: torch.Size([5544, 1, 1024])


In [4]:
if create_new_model:
    model = et.train_model( model=model_net,
                            epochs=epochs,
                            ecg_noisy=ecg_noisy,
                            ecg_clean=ecg_clean)
else:
    model = et.load_model( model_name=load_model_name,
                            model=model_net)

Loading model model_2021-3-15_819.pt. (make sure name ends in .pt)
Model model_2021-3-15_819.pt has been loaded


In [5]:
if create_new_model == True or create_new_res == True:
    ecg_res = et.concat_pt_full_cnn(model, ecg_noisy)
    et.get_local_time()
else:
    ecg_res = np.load( ecg_saved_res )

ecg_res = ecg_res.flatten()

Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (544, 1, 1024)
Complete shape is: (5544, 1, 1024)
Filename: res_pt_full_2021-3-15_819


In [6]:
ecg_noisy = torch.from_numpy(np.load('all_none_fs1024_en6.npy')).cuda().double()
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_noisy = et.norm_sig( ecg_noisy )

ecg_res = et.concat_pt_full_cnn(model, ecg_noisy)
ecg_res = torch.from_numpy(ecg_res).cuda()
ecg_res = ecg_res.flatten()

with torch.cuda.device(0):
    evals = eva.get_eval_metrics(ecg_clean=ecg_clean.flatten(), ecg_noisy=ecg_noisy.flatten(), ecg_denoised=ecg_res)

Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (544, 1, 1024)
Complete shape is: (5544, 1, 1024)
Filename: res_pt_full_2021-3-15_819
SNR Improvement: 2.3747526383825255
RMSE: 0.3907016224562534
PRD: 7.117399755214856
Elapsed time: 1067.47, (in mins: 17:47)


In [5]:
ecg_noisy = torch.from_numpy(np.load('all_none_fs1024_e00.npy')).cuda().double()
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_noisy = et.norm_sig( ecg_noisy )

ecg_res = et.concat_pt_full_cnn(model, ecg_noisy, 'res_CNN_n6_00')
ecg_res = torch.from_numpy(ecg_res).cuda()
ecg_res = ecg_res.flatten()

with torch.cuda.device(0):
    evals = eva.get_eval_metrics(ecg_clean=ecg_clean.flatten(), ecg_noisy=ecg_noisy.flatten(), ecg_denoised=ecg_res)

Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (544, 1, 1024)
Complete shape is: (5544, 1, 1024)
Filename: res_CNN_n6_00
SNR Improvement: 1.5255433551445228
RMSE: 0.3890334042583899
PRD: 7.087009874265621
Elapsed time: 1149.02, (in mins: 19:9)


In [6]:
ecg_noisy = torch.from_numpy(np.load('all_none_fs1024_e06.npy')).cuda().double()
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_noisy = et.norm_sig( ecg_noisy )

ecg_res = et.concat_pt_full_cnn(model, ecg_noisy, 'res_CNN_n6_06')
ecg_res = torch.from_numpy(ecg_res).cuda()
ecg_res = ecg_res.flatten()

with torch.cuda.device(0):
    evals = eva.get_eval_metrics(ecg_clean=ecg_clean.flatten(), ecg_noisy=ecg_noisy.flatten(), ecg_denoised=ecg_res)

Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (544, 1, 1024)
Complete shape is: (5544, 1, 1024)
Filename: res_CNN_n6_06
SNR Improvement: -0.6259723872149108
RMSE: 0.39017563333493227
PRD: 7.107817827144527
Elapsed time: 1149.40, (in mins: 19:9)


In [7]:
ecg_noisy = torch.from_numpy(np.load('all_none_fs1024_e12.npy')).cuda().double()
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_noisy = et.norm_sig( ecg_noisy )

ecg_res = et.concat_pt_full_cnn(model, ecg_noisy, 'res_CNN_n6_06')
ecg_res = torch.from_numpy(ecg_res).cuda()
ecg_res = ecg_res.flatten()

with torch.cuda.device(0):
    evals = eva.get_eval_metrics(ecg_clean=ecg_clean.flatten(), ecg_noisy=ecg_noisy.flatten(), ecg_denoised=ecg_res)

Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (1000, 1, 1024)
Result size: (544, 1, 1024)
Complete shape is: (5544, 1, 1024)
Filename: res_CNN_n6_06
SNR Improvement: -5.056749601772749
RMSE: 0.39422894623945776
PRD: 7.1816569069341405
Elapsed time: 1183.09, (in mins: 19:43)


In [5]:
ecg_res = np.load('res_pt_full_2021-3-15_726.npy')

In [8]:
eva.qrs_detector(ecg_clean.cpu().numpy().flatten(), ecg_noisy.cpu().numpy().flatten(), ecg_res.flatten())

Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
